### Introduction
A beginner notebook that explores methods to predict temperatures using machine learning. (work in progress :))

### Importing Libraries and Data
Importing libraries and cleaning the data.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import RobustScaler
import joblib
import csv
import datetime
from sklearn.svm import SVR
import sklearn.svm as svm
from sklearn.linear_model import LinearRegression

In [ ]:
fields = ['MM/DD/YYYY','YEAR','MONTH','DAY','TMAX','TMIN']
df = pd.read_csv("../input/new-york-city-daily-temperature-18692021/nyc_temp_1869_2021.csv", usecols=fields)
df.dropna()

df['TMAX'] /= 10
df['TMAX'] = (df['TMAX'] * 1.8) + 32
df['TMIN'] /= 10
df['TMIN'] = (df['TMIN'] * 1.8) + 32

df['MEAN'] = df[['TMAX', 'TMIN']].astype(float).mean(axis=1)
df['MM/DD/YYYY'] = pd.to_datetime(df['MM/DD/YYYY'])

df.tail()

### TODO: Exploratory Analysis

### Split data into test/training sets
Splitting time series data sets along the sequences. (Could use TimeSeriesSplit as well)

In [ ]:
data1 = df[(df['MM/DD/YYYY'] > '1869-01-01') & (df['MM/DD/YYYY'] < '2019-01-01')]
data2 = df[(df['MM/DD/YYYY'] > '2019-01-01') & (df['MM/DD/YYYY'] < '2021-05-04')]

data1 = data1.rename(columns={"MM/DD/YYYY":"DATE"})
data2 = data2.rename(columns={"MM/DD/YYYY":"DATE"})

data1['DATE2']=pd.DatetimeIndex(data1.DATE) + pd.DateOffset(1)
data2['DATE2']=pd.DatetimeIndex(data2.DATE) + pd.DateOffset(1)

### Train Data
Preparing matrices for the model.

In [ ]:
x = data1.drop('YEAR', axis = 1)
x = x.drop('MONTH', axis = 1)
x = x.drop('DAY', axis = 1)
x = x.drop('TMAX', axis = 1)
x = x.drop('TMIN', axis = 1)
y = x.MEAN
X = x.drop('MEAN', axis= 1)

x2 = data2.drop('YEAR', axis = 1)
x2 = x2.drop('MONTH', axis = 1)
x2 = x2.drop('DAY', axis = 1)
x2 = x2.drop('TMAX', axis = 1)
x2 = x2.drop('TMIN', axis = 1)
y2 = x2.MEAN
X2 = x2.drop('MEAN', axis= 1)

new_dates = []
counter = 0
X = X.append(X2)
dates = X.DATE

for day in dates:
    day = datetime.datetime.strptime(str(day), "%Y-%m-%d %H:%M:%S")
    day2 = (day - datetime.datetime(1970,1,1)).total_seconds()
    new_dates.append(day2)
    
X.DATE = new_dates
new_dates= []
for day in X.DATE2:
    day = datetime.datetime.strptime(str(day), "%Y-%m-%d %H:%M:%S")
    day2 = (day - datetime.datetime(1970,1,1)).total_seconds()
    new_dates.append(day2)
X.DATE2 = new_dates
y = y.append(y2)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.5, 
                                                    random_state=123, 
                                                    )

Preprosses training set using random forest regressor, declaring hyperparameters for tuning, then run linear regression model for prediction.

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

pipeline = make_pipeline(preprocessing.StandardScaler(), 
                     RandomForestRegressor(n_estimators=100))

hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
              'randomforestregressor__max_depth': [None, 5, 3, 1], }

clf = LinearRegression()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

joblib.dump(clf, 'weather_predictor.pkl')

### Predict Tomorrow's Weather
Add in dates to predict tomorrow's weather!

In [ ]:
clf = joblib.load('weather_predictor.pkl')
today = datetime.datetime(2021, 5, 6)
tomorrow = datetime.datetime(2021, 5, 7)

day = datetime.datetime.strptime(str(today), "%Y-%m-%d %H:%M:%S") ## date_list[0]
date = (day - datetime.datetime(1970,1,1)).total_seconds()
day_x = datetime.datetime.strptime(str(tomorrow), "%Y-%m-%d %H:%M:%S") ## date_list_2[0]
date_x = (day_x - datetime.datetime(1970,1,1)).total_seconds()

X = [[date, date_x]]
pred = clf.predict(X)[0]
print('predicted temperature for {}: '.format(str(tomorrow.strftime("%Y/%m/%d"))), pred)